**Task 08: Completing missing data**

In [ ]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4/"

     |████████████████████████████████| 235kB 2.7MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 


In [29]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib.namespace import RDF, RDFS
g1 = Graph()
g2 = Graph()
g1.parse(github_storage+"resources/data01.rdf", format="xml")
g2.parse(github_storage+"resources/data02.rdf", format="xml")

<Graph identifier=Nbb58ab7ab9e843c9b0fc610be892de82 (<class 'rdflib.graph.Graph'>)>

Task: list all the elements of the class Person in the first graph (data01.rdf) and complete the fields (given name, family name and email) that may be missing with the data from the second graph (data02.rdf). You can use SPARQL queries or iterate the graph, or both.

1. List all the elements of Person in data01.rdf

In [30]:
from rdflib.plugins.sparql import prepareQuery
ns = Namespace("http://data.org#")
q1 = prepareQuery('''
  SELECT 
    ?s
  WHERE { 
    ?s a/rdfs:subClassOf* ns:Person. 
  }
  ''',
  initNs = { "ns": ns}
)

for r in g1.query(q1):
  print(r)

(rdflib.term.URIRef('http://data.org#SaraJones'),)
(rdflib.term.URIRef('http://data.org#JohnDoe'),)
(rdflib.term.URIRef('http://data.org#JohnSmith'),)
(rdflib.term.URIRef('http://data.org#HarryPotter'),)


1.1. List all the elements of Person in data02.rdf (just in case)

In [32]:
for r in g2.query(q1):
  print(r)

(rdflib.term.URIRef('http://data.org#JohnSmith'),)
(rdflib.term.URIRef('http://data.org#SaraJones'),)
(rdflib.term.URIRef('http://data.org#JohnDoe'),)
(rdflib.term.URIRef('http://data.org#HarryPotter'),)


2.1. See the fields (given name, family name and email) in data01.rdf

In [33]:
q2 = prepareQuery('''
  SELECT 
    ?s ?p
  WHERE { 
    ?s a/rdfs:subClassOf* ns:Person. 
    ?s ?p ?o.
    FILTER(?p != rdf:type).
  }
  ''',
  initNs = { "ns": ns, "rdf": RDF}
)

for r in g1.query(q2):
  print(r.s, r.p)

http://data.org#JohnDoe http://www.w3.org/2001/vcard-rdf/3.0#EMAIL
http://data.org#JohnDoe http://www.w3.org/2001/vcard-rdf/3.0#Family
http://data.org#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0#Given
http://data.org#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0#EMAIL
http://data.org#HarryPotter http://www.w3.org/2001/vcard-rdf/3.0#Given
http://data.org#HarryPotter http://www.w3.org/2001/vcard-rdf/3.0#Family


2.2. Complete the fields (given name, family name and email) that may be missing with the data from data02.rdf

We need to add to g1 the fields missing. First, we obtain all the properties that g2 has and then check if that field is filled in g1.

In [34]:
for person,y,z in g2.triples((None, RDF.type, ns.Person)): #All the elements of Person in g2
  for x, prop, obj in g2.triples((person, None, None)): #All the properties of those people
    if prop!=RDF.type and (person, prop, obj) not in g1.triples((person,None,None)): #Check if that property is already in g1
      print(person, prop, obj)
      g1.add((person, prop, obj))

http://data.org#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0#Family Smith
http://data.org#SaraJones http://www.w3.org/2001/vcard-rdf/3.0#Family Jones
http://data.org#SaraJones http://www.w3.org/2001/vcard-rdf/3.0#EMAIL sara.jones@data.org
http://data.org#SaraJones http://www.w3.org/2001/vcard-rdf/3.0#Given Sara
http://data.org#JohnDoe http://www.w3.org/2001/vcard-rdf/3.0#Given John
http://data.org#HarryPotter http://www.w3.org/2001/vcard-rdf/3.0#FN Harry Potter
http://data.org#HarryPotter http://www.w3.org/2001/vcard-rdf/3.0#EMAIL hpotter@hogwarts.org


TAKE INTO ACCOUNT: the person called "Harry Potter" has an extra field called "FN" in g2. It has been added to g1.

In [35]:
print(g1.serialize(format="xml").decode("UTF-8"))

<?xml version="1.0" encoding="UTF-8"?>
<rdf:RDF
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
   xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
   xmlns:vcard-rdf="http://www.w3.org/2001/vcard-rdf/3.0#"
>
  <rdf:Description rdf:about="http://data.org#JohnSmith">
    <vcard-rdf:Given rdf:datatype="http://www.w3.org/2001/XMLSchema#string">John</vcard-rdf:Given>
    <rdf:type rdf:resource="http://data.org#Person"/>
    <vcard-rdf:Family rdf:datatype="http://www.w3.org/2001/XMLSchema#string">Smith</vcard-rdf:Family>
    <vcard-rdf:EMAIL rdf:datatype="http://www.w3.org/2001/XMLSchema#string">j.smith@data.org</vcard-rdf:EMAIL>
  </rdf:Description>
  <rdf:Description rdf:about="http://data.org#SaraJones">
    <rdf:type rdf:resource="http://data.org#Person"/>
    <vcard-rdf:Family rdf:datatype="http://www.w3.org/2001/XMLSchema#string">Jones</vcard-rdf:Family>
    <vcard-rdf:Given rdf:datatype="http://www.w3.org/2001/XMLSchema#string">Sara</vcard-rdf:Given>
    <vcard-rdf:EMAIL